# NewEgg.Com WebScraping Program For Laptops - Beta

###  - April 2020

---

In [1]:
# Import dependencies
import os
import glob
import time
import datetime
import re
from re import search
import pandas as pd
from splinter import Browser
import requests
from bs4 import BeautifulSoup as soup
import random

## Functions & Classes Setup
---

In [2]:
# Build a function to return data, which will be used throughout the program
def return_dt():
    global current_date
    current_date = str(datetime.datetime.now()).replace(':','.').replace(' ','_')[:-7]
    return current_date
#return_dt()

In [3]:
"""
Created a function to ask users if they want to watch the Bot (headless = False) work OR not (headless = True)
# Lastly, will take you directly to the webpage that was inputted
"""
#head = ''
#browser =''
def head_on_off(executable_path):
    # Have moved two preset variables, head and browser that are both " = '' "
    # assigning these as global variables enable us to reference them outside and inside the function
    global head
    global browser
    # options creates a bound to an answer
    options = [1, 2]
    #executable_path = {'executable_path': './chromedriver.exe'}
    # for all cases where users input in a value that is not valid
    while head not in options:
        head = int(input('Do you want the desktop watch the bot work? Enter a number: 1 - YES | 2 - NO . Your Answer: '))
        if head not in options:
            print("That was not a valid answer. Please try again. ")
    # For cases where users enter in valid options:
    if head == options[0]:
        print('Head is activated. Please view only the new automated Google Chrome web browser. ')
        print('Do not make any adjustments to this automated window while the program runs, as it may produce errors or undesired outputs. Only time you should touch the automated window when the recaptcha ask you to prove you are a human. ')
        browser = Browser('chrome', **executable_path, headless=False, incognito=True)
    if head == options[1]:
        print('Headless mode activated. No web browser will pop up. Please proceeed. ')
        browser = Browser('chrome', **executable_path, headless=True)
    # visit the target site
    browser.visit(url)
#     global current_url
    current_url = browser.url
    #print(current_url)
    
    return current_url

#head_on_off(executable_path)
#time.sleep(5)

In [4]:
"""
Step 5) Previews to the user of the scrappable items-contrainers on the page

"""
def scrappable_y_n(current_page_soup):
    global containers
    containers = current_page_soup.find_all("div", class_="item-container")
    
    # print first and last objects so users can understand what the output will be
    print("Preview: expect these scrapped off this page, and for every other total results pages, if there's more than one: ")
    print("="*35)
    # max items should be 36
    counter = 0
    for con in containers:
        try:
            counter += 1
            product_details = con.find_all("a", class_="item-title")[0].text
            product_price = con.find_all("li", class_="price-current")[0].text.split()[0]
            print(f'{counter}) {product_details} | Price: {product_price}')
            print("-"*35)
            
        except (IndexError) as e:
            print(f"{counter}) This item was not scrappable. Skipped. ")
            print("-"*35)
            
    print("="*60)
    if counter == 0:
        print("Unable to scrap this link. ")
    else:
        print(f"{len(containers)} Scrappable Objects on the page. ")
    #return current_page_soup
#scrappable_y_n(current_page_soup)

In [5]:
"""
Originally modeled out parent/child inheritance object structure.

After careful research, I found it much easier to export the Pandas Dataframe of the results to a dictionary, and then into a class object, which I will elaborate more down below.
"""
# class Product_catalog:
    
#     all_prod_count = 0
    
#     def __init__(self, general_category): # computer systems
#         self.general_category = general_category
        
#         Product_catalog.all_prod_count += 1
        
#     def count_prod(self):
#         return int(self.all_prod_count)
#         #return '{}'.format(self.general_category)
        
# class Sub_category(Product_catalog): # laptops/notebooks, gaming
    
#     sub_category_ct = 0
    
#     def __init__(self, general_category, sub_categ, item_num, brand, price, img_link, prod_link, model_specifications, current_promotions):
#         super().__init__(general_category)
#         Sub_category.sub_category_ct += 1
        
#         self.sub_categ = sub_categ
#         self.item_num = item_num
#         self.brand = brand
#         self.price = price
#         self.img_link = img_link
#         self.prod_link = prod_link
#         self.model_specifications = model_specifications
#         self.current_promotions = current_promotions

'\nOriginally modeled out parent/child inheritance object structure.\n\nAfter careful research, I found it much easier to export the Pandas Dataframe of the results to a dictionary, and then into a class object, which I will elaborate more down below.\n'

In [6]:
"""
Main NewEgg WebScraper function.

"""
def newegg_page_scraper(containers, turn_page): #before: (containers, turn_page)
    page_nums = []
    general_category = []
    product_categories = []
    images = []
    product_brands = []
    product_models = []
    product_links = []
    item_numbers = []
    promotions = []
    prices = []
    shipping_terms = []
    
    # Put this to avoid error that was being generated
    global gen_category
    
    """ 
    Loop through all the containers on the HTML, and scrap the following content into the following lists
    
    """
    for con in containers:
        try:
            page_counter = turn_page
            page_nums.append(int(turn_page))
            
            gen_category = target_page_soup.find_all('div', class_="nav-x-body-top-bar fix")[0].text.split('\n')[5]
            general_category.append(gen_category)
            
            prod_category = target_page_soup.find_all('h1', class_="page-title-text")[0].text
            product_categories.append(prod_category)
            
            image = con.a.img["src"]
            #print(image)
            images.append(image)

            prd_title = con.find_all('a', class_="item-title")[0].text
            product_models.append(prd_title)

            product_link = con.find_all('a', class_="item-title")[0]['href']
            product_links.append(product_link)
            
            shipping = con.find_all('li', class_='price-ship')[0].text.strip().split()[0]
            if shipping != "Free":
                shipping = shipping.replace('$', '')
                shipping_terms.append(shipping)
            else:
                shipping = 0.00
                shipping_terms.append(shipping)

            brand_name = con.find_all('a', class_="item-brand")[0].img["title"]
            product_brands.append(brand_name)

        except (IndexError, ValueError) as e:
            # if there's no item_brand container, take the Brand from product details
            product_brands.append(con.find_all('a', class_="item-title")[0].text.split()[0])
            #print(f"{e} block 1")

        try:
            current_promo = con.find_all("p", class_="item-promo")[0].text
            promotions.append(current_promo)
        except:
            promotions.append('null')
            #print(f"{e} block 2")
        try:
            price = con.find_all('li', class_="price-current")[0].text.split()[0].replace('$','').replace(',', '')
            prices.append(price)
        except:
            price = 'null / out of stock'
            prices.append(price)
            #print(f"{e} block 3")
        
        try:
            item_num = con.find_all('a', class_="item-title")[0]['href'].split('p/')[1].split('?')[0]
            item_numbers.append(item_num)
        except (IndexError) as e:
            item_num = con.find_all('a', class_="item-title")[0]['href'].split('p/')[1]
            item_numbers.append(item_num)    
    
    # Convert all of the lists into a dataframe
    df = pd.DataFrame({
    'item_number': item_numbers,
    'general_category': general_category,
    'product_category': product_categories,
    'brand': product_brands,
    'model_specifications': product_models,
    'price': prices,
    'current_promotions': promotions,
    'shipping': shipping_terms,
    'page_number': page_nums,
    'product_links': product_links,
    'image_link': images
    })
    
    # Rearrange the dataframe columns into the following order
    df = df[['item_number', 'general_category','product_category', 'page_number' ,'brand','model_specifications' ,'current_promotions' ,'price' ,'shipping' ,'product_links','image_link']]
    
    # Convert the dataframe into a dictionary
    global scraped_dict
    scraped_dict = df.to_dict('records')
    
    # Grab the subcategory "Laptop/Notebooks" and eliminate any special characters that may cause errors
    global pdt_category
    pdt_category = df['product_category'].unique()[0]
    # eliminate special characters in a string if it exists
    pdt_category = ''.join(e for e in pdt_category if e.isalnum())
    
    """ Count the number of items scraped by getting the length of a all the models for sale.
        This parameter is always available for each item-container in the HTML
    """
    global items_scraped
    items_scraped = len(df['model_specifications'])
    
    """
    Save the results into a csv file using Pandas
    """
    df.to_csv(f'./processing/{current_date}_{pdt_category}_{items_scraped}_scraped_page{turn_page}.csv')
    
    # Return these variables as they will be used.
    return scraped_dict, items_scraped, pdt_category
    
#df.head()
#newegg_page_scraper(containers, turn_page)

In [7]:
# create a function to return the total results pages, if exists, otherwise just scrape one page

def results_pages(current_page_soup):
    # Use BeautifulSoup to extract the total results page number
    results_pages = current_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()
    #print(results_pages)
    # Find and extract total pages + and add 1 to ensure proper length of total pages
    global total_results_pages
    total_results_pages = int(re.split("/", results_pages)[1]) + 1 # need to add 1 b/c 'range(inclusive, exclusive)'
    #========================================= need to remember to +2, and remove -30
    #print(total_results_pages)
    
    return total_results_pages
#results_pages()

In [8]:
"""
Build a function to concatenate all pages that were scraped and saved in the processing folder.
Save the final output (1 csv file) all the results
    
"""
def concatenate(total_results_pages):
    
    path = f'./processing\\'
    scraped_pages = glob.glob(path + "/*.csv")
    concatenate_pages = []
    counter = 0
    for page in scraped_pages:
        df = pd.read_csv(page, index_col=0, header=0)
        concatenate_pages.append(df)

    compiled_data = pd.concat(concatenate_pages, axis=0, ignore_index=True)
    total_items_scraped = len(compiled_data['brand']) # can replace this counter by creating class objects everytime it scrapes
    concatenated_output = compiled_data.to_csv(f"./finished_outputs/{current_date}_{total_items_scraped}_scraped_{total_results_pages}_pages_.csv")
    return concatenated_output

In [9]:
"""
Built a function to clear out the entire processing files folder to avoid clutter.
Or the user can keep the processing files (page by page) for their own analysis.

"""
def clean_processing_fldr():
    # delete all files in the 'processing folder'
    path = f'./processing\\'
    scraped_pages = glob.glob(path + "/*.csv")
    if len(scraped_pages) < 1:
        print("There are no files in the folder to clear. ")
    else:
        print(f"Clearing out a total of {len(scraped_pages)} scraped pages in the processing folder... ")
        clear_processing_files = []
        for page in scraped_pages:
            os.remove(page)
        
    print('Clearing of "Processing" folder complete. ')

In [10]:
def rdm_slp_4_20():
    x = random.randint(4, 20)
    time.sleep(x)
    print(f"Slept for {x} seconds. ")
    

In [11]:
""" This class takes in the dictionary from the webscraper function, and will be used in a list comprehension to produce class "objects" in the "product_catalog"

"""
class Laptops:
    def __init__(self, **entries):
        self.__dict__.update(entries)

## Main Program Logic
---

In [12]:
""" Welcome to the program message!
"""

print("=== NewEgg.Com WebScraper Beta ===")
print("=="*30)
print('Scope: This project is a beta and is only built to scrape the laptop section of NewEgg.com due to limited time. ')
print("")
print("Instructions:")
print("")
return_dt()
print(f'Current Date And Time: {current_date}')
print("")
print("(1) Go to www.newegg.com, go to the laptop section, select your requirements (e.g. brand, screensize, and specifications - SSD size, processor brand and etc...) ")
print("(2) Copy and paste the url from your exact search when prompted ")
print('(3) Activate or Disable the "Head View", webscraper bots point of view ')
print('(4) After the webscraping is successful, you will have an option to concatenate all of the pages you scraped together into one csv file')
print('(5) Lastly, you will have an option to clear out the processing folder (data scraped by each page)')
print('(6) If you have any issues or errors, "PRESS CTRL + C" to quit the program in the terminal ')
print('(7) Disclaimer: Newegg may ban you for a 24 - 48 hours for webscraping their data, then you may resume ')
print("")

# Set up Splinter requirements
executable_path = {'executable_path': './chromedriver.exe'}

# Ask user to input in the laptop query link they would like to scrape
url = input("Step 1) Please copy and paste your laptop query that you want to webscrape, and press enter: ")
head = ''
browser =''
head_on_off(executable_path)
# Sometimes it will ask if the user is a bot. After the user answers the questions, then they may enter "y" to proceed
safe_proceed_y_n = input(f'The Break Pedal: Answer any robot queries by NewEgg. Enter "y" when you are ready to proceed. ')
if safe_proceed_y_n == 'y':
    print(f'Proceeding with webscrape... ')
    current_url = browser.url
else:
    print("Quitting browser. You will need to press CTRL + C to QUIT the program, and then restart it to try scraping the link again. ")
    browser.quit()

# head_on_off will finish with browse.visit(url) - and returns "current_url" as an output that passed into response
response = requests.get(current_url)
current_page_soup = soup(response.text, 'html.parser')

# might need to delete - unnecessary
##current_page_soup.find_all("div", class_="item-container")

# Preview some of the data that will be scraped
scrappable_y_n(current_page_soup)

# Run the results_pages function to gather the total pages to be scraped
results_pages(current_page_soup)

time.sleep(2)

product_catalog = []
# This is the loop that sets up the page by page scraping of data / results of the user's query
for turn_page in range(1, total_results_pages):
    
    # check if 
    
    # Goal here is to emulate human behavior by clicking on the screen
    browser.find_by_tag('h1')[0].click()
    # want to mimic human behavior by clicking on at least one link and fiewing for 6 seconds
    number_of_a_tags = len(browser.find_by_tag('a'))
    ninety_pct_a_tag = int(round((number_of_a_tags * .905)))
    ninety_98th_a_tag = int(round((number_of_a_tags * .98)))
    
    try:
        rdm_slp_4_20()
        browser.find_by_tag('h1')[0].click()
        # create a random generator loop for multiple mouse overs on the screen
            
        browser.find_by_tag('a')[ninety_pct_a_tag].click()
        browser.back()
    except:
        browser.find_by_tag('h1')[0].click()
        browser.find_by_tag('a')[ninety_pct_a_tag + 1].click()
        rdm_slp_4_20()
        browser.back()

# Might need to delete these below:
#     # set the current url as the target page (aiming the boomerang)
#     target_url = browser.url

#     # Use Request.get() - throw the boomerang at the target, retrieve the info, & return back to requestor
#     response_target = requests.get(target_url)
#     #response

#     # Use BeautifulSoup to read grab all the HTML using the lxml parser
#     target_page_soup = soup(response_target.text, 'html.parser')

##############################

    # if "reCAPTCHA", pauses the program, and allows the user to continue to scrape after they're done completing the quizz - move to beginning of the loop and also include an OR
    
    # package up the 
    if browser.is_element_present_by_id('g-recaptcha') == True:
        continue_scraper = input("Noticed Newegg asked if you were robot. When you're done proving you are not a robot and completing all picture tests, enter in any key and press ENTER to continue the webscrape. ")
        
        # target the new page's url for scraping
        target_url = browser.url

        # Use Request.get() - throw the boomerang at the target, retrieve the info, & return back to requestor
        response_target = requests.get(target_url)
        #response

        # Use BeautifulSoup to read grab all the HTML using the lxml parser
        target_page_soup = soup(response_target.text, 'html.parser')
        containers = target_page_soup.find_all("div", class_="item-container")
        newegg_page_scraper(containers, turn_page)
    
        # After scraping a page of data, create instances of objects of the laptops/notebooks using a list comprehension
        objects = [Laptops(**prod_obj) for prod_obj in scraped_dict]

        # Append all of the objects off the page to the main product_catalog list (List of List of Objects by page scraped)
        product_catalog.append(objects)
        # Click to the next page
        print("Will scrape pages, but will need to randomly sleep for max 35 seconds to emulate human behavior. ")
        try:
            x = random.randint(8, 28)
            print(f"{turn_page}) | SLEEPING FOR {x} SECONDS ")
            time.sleep(x)
            browser.find_by_tag('h1')[0].click()
            time.sleep(3)
            browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()

        except:
            x = random.randint(7, 25)
            print(f"{turn_page}) | SLEEPING FOR {x} SECONDS - EXCEPTION")
            time.sleep(x)
            browser.find_by_tag('h1')[0].click()
            browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[4]/div/div/div[10]/button').click()
            time.sleep(1)
        
        browser.quit()
        
#######################################        
        
    else:
        time.sleep(3)
        # set the current url as the target page (aiming the boomerang)
        target_url = browser.url

        # Use Request.get() - throw the boomerang at the target, retrieve the info, & return back to requestor
        response_target = requests.get(target_url)
        #response

        # Use BeautifulSoup to read grab all the HTML using the lxml parser
        target_page_soup = soup(response_target.text, 'html.parser')


        # Use BeautifulSoup to extract the total results page number
        #results_pages = target_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()

        # Run the webscraper function by passing in containers and turn_page
        containers = target_page_soup.find_all("div", class_="item-container")
        newegg_page_scraper(containers, turn_page)

        # After scraping a page of data, create instances of objects of the laptops/notebooks using a list comprehension
        objects = [Sub_category(**i) for i in scraped_dict]
        
        # Append all of the objects off the page to the main product_catalog list (List of List of Objects by page scraped)
        product_catalog.append(objects)

        # Have the scraper randomly sleep for min of 3 - 19 seconds, then flip the page.

        # Click to the next page
        try:
            x = random.randint(8, 27)
            print(f"{turn_page}) | SLEEPING FOR {x} SECONDS ")
            time.sleep(x)
            browser.find_by_tag('h1')[0].click()
            #browser.find_by_tag('body')[0].click()
            time.sleep(4)
            browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()

        except:
            x = random.randint(7, 35)
            print(f"{turn_page}) | SLEEPING FOR {x} SECONDS - EXCEPTION")
            time.sleep(x)
            browser.find_by_tag('h1')[0].click()
            time.sleep(4)
            browser.find_by_tag('h1')[0].click()
            time.sleep(1)
            browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[4]/div/div/div[10]/button').click()


# Exit the broswer once complete webscraping
browser.quit()

# Prompt the user if they would like to concatenate all of the pages into one csv file
concat_y_n = input(f'All {total_results_pages} pages have been saved in the "processing" folder (1 page = csv files). Would you like for us concatenate all the files into one? Enter "y", if so. Otherwise, enter anykey to exit the program. ')

if concat_y_n == 'y':
    concatenate(total_results_pages)
    print(f'WebScraping Complete! All {total_results_pages} have been scraped and saved as {current_date}_{pdt_category}_scraped_{total_results_pages}_pages_.csv in the "finished_outputs" folder')

# Prompt the user to if they would like to clear out processing folder function here - as delete everything to prevent clutter
clear_processing_y_n = input(f'The "processing" folder has {total_results_pages} csv files of each page that was scraped. Would you like to clear the files? Enter "y", if so. Otherwise, enter anykey to exit the program. ')
if clear_processing_y_n == 'y':
    clean_processing_fldr()

print('Thank you checking out my project, and hope you found this useful!')

=== NewEgg.Com WebScraper Beta ===
Scope: This project is a beta and is only built to scrape the laptop section of NewEgg.com due to limited time. 

Instructions:

Current Date And Time: 2020-04-16_21.38.17

(1) Go to www.newegg.com, go to the laptop section, select your requirements (e.g. brand, screensize, and specifications - SSD size, processor brand and etc...) 
(2) Copy and paste the url from your exact search when prompted 
(3) Activate or Disable the "Head View", webscraper bots point of view 
(4) After the webscraping is successful, you will have an option to concatenate all of the pages you scraped together into one csv file
(5) Lastly, you will have an option to clear out the processing folder (data scraped by each page)
(6) If you have any issues or errors, "PRESS CTRL + C" to quit the program in the terminal 
(7) Disclaimer: Newegg may ban you for a 24 - 48 hours for webscraping their data, then you may resume 

Step 1) Please copy and paste your laptop query that you want

Slept for 4 seconds. 
1) | SLEEPING FOR 22 SECONDS 
Slept for 11 seconds. 


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=80.0.3987.163)


In [ ]:
#/html/body/div[5]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button


In [17]:
print(browser.find_by_tag('a')[ninety_pct_a_tag].__dict__)

{'_element': <selenium.webdriver.remote.webelement.WebElement (session="92cc9c9233e821dc38ef74d6f05ca03d", element="72ac5fdf-5076-4495-b827-c66e93c44d33")>, 'parent': <splinter.driver.webdriver.chrome.WebDriver object at 0x000001F3B63C64E0>}


In [18]:
len(browser.find_by_tag('a')[ninety_pct_a_tag])

TypeError: object of type 'WebDriverElement' has no len()

In [ ]:
mimipopupsweepstakes

In [ ]:
#browser.find_by_tag('button')#[0].click()

In [ ]:
len(current_page_soup.find_all("div", class_="item-container"))

In [ ]:
#id="rc-imageselect"

In [ ]:
#browser.is_element_present_by_id("btn_InnerSearch")
#browser.is_element_present_by_id("rc-imageselect")

In [ ]:
print(browser.__dict__)

In [ ]:
browser.find_by_tag('body')[0].click()

In [ ]:
len(browser.find_by_tag('body'))

In [21]:
browser.find_by_tag('a')[1958].click()#['href']

In [20]:
browser.back()

In [ ]:
number_of_a_tags = len(browser.find_by_tag('a'))
ninety_pct_a_tag = round((number_of_a_tags * .902))
ninety_pct_a_tag

In [ ]:
browser.find_by_tag('a')[1962].click()

In [26]:
browser.find_by_tag('a')[1970].mouse_over()

In [ ]:
browser.find_by_tag('h1')[0].click()

In [ ]:
#//*[@id="btn_InnerSearch"]

In [ ]:
browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()

In [ ]:
browser.is_element_present_by_id('g-recaptcha')

In [ ]:
browser.is_element_present_by_id('mimipopupsweepstakes')


In [ ]:
browser.find_by_tag('a')[2263].click()

In [ ]:
browser.is_element_present_by_id('sweepstakespopup')
/html/body/div[16]/div/a

In [ ]:
browser.is_element_present_by_id('sweepstakespopup_close')

In [ ]:
browser.find_by_id('sweepstakespopup_close').click()

In [ ]:
# browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[4]/div/div/div[10]/button').click()

In [ ]:
#current_page_soup.is_element_present_by_xpath("/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button")


In [ ]:
#is_element_present_by_xpath('')

In [ ]:
# Test to view objects that were created from the data we collected
#product_catalog[1][0].__dict__

In [ ]:
#/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button/i

In [ ]:
#/html/body/div[4]/section/div/div/div[2]/div/div/div[2]/div[2]/div[1]/div[2]/div[4]/div/div/div[11]/button/i

In [ ]:
counter = 0
prices = []
for con in containers:
    counter += 1
    try:
        #print(str(counter) + " | " + con.find_all('li', class_="price-current")[0].text.split()[0].replace('$','').replace(',', ''))
        price = con.find_all('li', class_="price-current")[0].text.split()[0].replace('$','').replace(',', '')
        prices.append(price)
    except:
        #print(str(counter) + " | " + "null")
        
        price = 'null'
        prices.append(price)
        
print(prices)